Краулер для nplus1

In [1]:
# Импортируем необходимые библиотеки:
import requests # http-запросы,
import re # регулярные выражения,
from bs4 import BeautifulSoup # удаление тегов html,
from tqdm import tqdm # красотуля для анализа прогресса.
import time
import random


In [2]:
# Функия принимает адрес статьи на nplus1.ru и возвращает текст статьи и метаинформацию по ней.
def getArticleTextNPlus1(adr):
    r = requests.get(adr) # Получаем текст статьи, ...
    tables=re.split("</div>",re.split('="tables"', r.text)[1])[0] # ... берем блок с метаинформацией, ...

    t1=re.split("</time>", re.split("<time", tables)[1])[0]
    n_time=re.split("</span>", re.split("<span>", t1)[1])[0]
    n_date=re.split("</span>", re.split("<span>", t1)[2])[0]
    n_rubr=re.split(">", re.split("</a>", re.split("<a href", tables)[1])[0])[1]
    n_diff=re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
# Достаем заголовок, автора и текст статьи.
    n_head=re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    n_author=re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    n_text=re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

# "Откусываем" оставшиеся теги.
    beaux_text=BeautifulSoup(n_text, "lxml")
    n_text=beaux_text.get_text() 
    time.sleep(random.uniform(1, 3))
# print(n_time, n_date, n_rubr, n_diff)
# print(n_head)
# print(n_author)
# print(n_text)
    return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
# def getArticleTextNPlus1(adr):

In [5]:
print(getArticleTextNPlus1('https://nplus1.ru/news/2015/10/29/radar'))

['16:47', '29 Окт. 2015', 'Оружие', '2.1', 'Василий Сычев', 'Французы испытали пассивный «летающий радар»', 'ВВС Франции провели испытания перспективной пассивной радиоэлектронной станции, предназначенной для установки на самолеты. Как\xa0сообщает\xa0Defense Aerospace, проверки системы проводились на 701-й авиабазе на юго-востоке Франции. Новый радар отличается от других пассивных систем возможностью обнаруживать воздушные цели по отраженным от них сигналам цифрового эфирного телевидения.Радар был установлен на планер Busard и провел в воздухе два часа. Во время этого полета проверялась возможность радара принимать и обрабатывать прямые и отраженные телевизионные сигналы. В ближайшее время планируется провести еще один испытательный полет, в ходе которого радара будет обнаруживать другие воздушные объекты.Новая радиолокационная станция разрабатывается для обнаружения аппаратов, летящих на малых скорости и высоте. Для обнаружения целей он может использовать не только отраженные сигналы 

In [3]:
hreffile=open(r"/home/mi_air/Downloads/nplus1/href_list_dedup.txt", "w", encoding="utf-8")

In [4]:
def getHrefs(link):

    # Получаем текст страницы, которая содержит ссылки на все статьи этого дня (в примере - 11.01.2016).
    r = requests.get(link)
    # Каждая ссылка на статью оформлена с помощью тега <article>
    refs=re.split('<article class="item item-news item-news', r.text)
    
    # Если ссылки вообще есть, ...
    if len(refs)>0:
        total=0
    # ..., то перебираем их все, кроме первой части - там только мусор. 
        for t in refs[1:]:
            href=re.split('"', t)[6]
            #print(href)
            hreffile.write("https://nplus1.ru"+href + "\n")

            #time.sleep(random.uniform(1, 4))
        


In [ ]:
link = "https://nplus1.ru/news/2017/01/28/"
#linkyear = int(link.split("/")[4])
#linkmonth = int(link.split("/")[5])
#linkday = int(link.split("/")[6])
for year in range(2014,2018):
    for month in range (1,13):
        if month <10:
            month = "0" + str(month)
        for day in range(1,32):
            if day<10:
                day = "0" + str(day)
            newlink = "https://nplus1.ru/news/" + str(year) +"/"+str(month)+"/"+str(day)+"/"
            print(newlink)
            try:
                getHrefs(newlink)
                
            except:
                pass


In [8]:
hreffile=open(r"/home/mi_air/Downloads/nplus1/href_list_dedup.txt", "r", encoding="utf-8")
hreffile2=open(r"/home/mi_air/Downloads/nplus1/href_list_dedup2.txt", "w", encoding="utf-8")
hreflist=[]
for line in hreffile:
    hreflist.append(line)
hreflist = list(set(hreflist))
print(len(hreflist))

7818


In [9]:
for line in hreflist:
    hreffile2.write(line)

In [5]:
authortext = open(r"/home/mi_air/Downloads/nplus1/href_list_dedup2.txt", "r", encoding="utf-8")
corpus = open(r"/home/mi_air/Downloads/nplus1/corpus_nplus1.txt", "w", encoding="utf-8")
#[n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
for line in authortext:
    params = getArticleTextNPlus1(line.strip("\n"))
    corpus.write("<author>"+params[4]+"\n")
    corpus.write("<textname>"+params[5]+"\n")
    corpus.write("<textdiff>"+params[3]+"\n")
    corpus.write("<textrubric>"+params[2]+"\n")
    corpus.write("<date>"+params[1]+"\n")
    corpus.write("<time>"+params[0]+"\n")
    corpus.write("<source>"+line+"\n")
    corpus.write("<textid>" + re.sub("/", "", re.split("https://nplus1.ru/news/",line)[1]) + "\n")
    corpus.write("<text>"+"\n")
    corpus.write(params[6]+"\n")
    corpus.write("</text>"+"\n")
    print(line)


https://nplus1.ru/news/2015/10/29/radar

https://nplus1.ru/news/2016/12/12/aggressive-drones

https://nplus1.ru/news/2015/08/26/twinjet-hubble

https://nplus1.ru/news/2015/08/26/shining-solved-profile

https://nplus1.ru/news/2016/01/11/snow

https://nplus1.ru/news/2016/04/04/rwanda

https://nplus1.ru/news/2015/10/07/makerarm-adam

https://nplus1.ru/news/2016/06/04/davd

https://nplus1.ru/news/2015/05/20/liquidantennas

https://nplus1.ru/news/2015/08/12/open-pixar

https://nplus1.ru/news/2016/03/18/cleaning

https://nplus1.ru/news/2016/06/07/power-glove

https://nplus1.ru/news/2016/10/05/pulltherope

https://nplus1.ru/news/2016/12/17/aquila

https://nplus1.ru/news/2015/07/15/avogadro

https://nplus1.ru/news/2016/11/22/-newdesalinationdevice

https://nplus1.ru/news/2015/08/18/first-flower

https://nplus1.ru/news/2015/06/16/many-faced-god

https://nplus1.ru/news/2015/11/19/solid-fuel

https://nplus1.ru/news/2016/07/11/si2-circumnavigatory

https://nplus1.ru/news/2016/02/17/flexible

https

In [4]:
line

'https://nplus1.ru/news/2015/10/29/radar\n'